In [ ]:
#! pip install openai

In [ ]:
# -------------------
# Imports
# -------------------

from openai import OpenAI
from pathlib import Path
import json
import time
import pandas as pd

# -------------------
# Global variables
# -------------------

# create a base prompt that will be used for all questions
BASE_PROMPT = """Below is a CSV file that contains titles of artworks. Each title is delineated by a comma. For each artwork title, give me the list of:
- object named entity
- location named entity
- person named entity
- genre named entity
- miscellaneous named entity.
For each title, format the output as an array of JSON literal objects with the following keys:
- TITLE for the title of the artwork
- OBJECT for object named entity
- LOCATION for location named entity
- PERSON for person named entity
- GENRE for genre named entity
- MISCELLANEOUS for miscellaneous named entity.
Example below: 
[
{
    "TITLE": "Portrait of the Artist Pablo Picasso with a Guitar in Paris",
    "OBJECT": ["guitar"],
    "LOCATION": ["Paris"],
    "PERSON": ["Pablo Picasso"],
    "GENRE": ["portrait"],
    "MISCELLANEOUS": ["artist"]
}
{
    "TITLE": "Strolling Musicians in an Inn Garden",
    "OBJECT": [],
    "LOCATION": ["Inn Garden"],
    "PERSON": ["Strolling Musicians"],
    "GENRE": [],
    "MISCELLANEOUS": []
}
]
"""

OPENAI_QUERY_PARAMS = {
    "model": "gpt-3.5-turbo",
    "temperature": 0,
    "max_tokens": 1024
}

# add your API key here
openai_key_filename = 'open_ai_api_key.txt'

home = str(Path.home()) # gets path to home directory; supposed to work for both Win and Mac
with open(home + '/' + openai_key_filename, 'r') as file:
    api_key_string = file.read().strip() # remove any leading or trailing white space or newlines

CLIENT = OpenAI(api_key=api_key_string)


In [ ]:
# -------------------
# Function definitions
# -------------------

def ask_openai(prompt: str, base_prompt=BASE_PROMPT, openai_query_params=OPENAI_QUERY_PARAMS) -> str:
    """Send a request to OpenAI's ChatGPT API to do entity recognition. The prompt should be a sentence or paragraph of text
    on which you want to perform NER.
    
    The function returns a JSON-formatted string with the named entities extracted from the input text.
    """
    response = CLIENT.chat.completions.create(
        messages=[
        {
            "role": "system", 
            "content": "You are a smart and intelligent Named Entity Recognition (NER) system whose job is to extract entities from the title of an artwork. You will look for people, location names, and common objects. You will also look for genres of visual art and any miscellaneous entities. The labels in your output should not include words that are not in the title."
        },
        {
            "role": "user", 
            "content": base_prompt + prompt
        }        
    ],
        **openai_query_params
    )
    
    return(response.choices[0].message.content)


#### NER on Titles: Paintings

In [ ]:
# getting artwork titles for NER
title_pd = pd.read_csv(home + '/GitHub/vandycite/gallery_buchanan/works_multiprop.csv')
painting_pd = pd.read_csv('/vision/painting_ner.csv')

# delete all columns except english label and inventory_number
title_pd = title_pd[['label_en', 'inventory_number']]

# only keep titles that are in the painting dataset
title_pd = title_pd[title_pd['inventory_number'].isin(painting_pd['inventory_number'])]

# isolate titles only to send to OpenAI
title_pd = title_pd['label_en']

# remove any titles that are "Untitled"
title_pd = title_pd[~title_pd.str.contains("Untitled")]

title_pd.shape

In [ ]:
ner_output_json = []

for i in range(0, len(title_pd), 15):
    # send 15 titles at a time to OpenAI
    title_pd_slice = title_pd[i:i+15]
    # call NER function and get output as json
    output = json.loads(ask_openai(title_pd_slice.to_csv(index=False, header=False)))
    # add output to ner_output_json list
    ner_output_json.extend(output)
    # track progress
    print(i)
    # sleep to avoid erroneous characters in output
    time.sleep(5)

In [ ]:
ner_output_df = pd.DataFrame(columns=['title', 'ner_output'])

# iterate through ner_output_json, extract title and ner labels to dataframe
for i in range(len(ner_output_json)):
    ner_output_df.at[i, 'title'] = ner_output_json[i]['TITLE']
    # add the keys that are not title as ner_output in the dataframe
    ner_labels = []
    for key in ner_output_json[i].keys():
        if key != 'TITLE' and key!= 'GENRE':
            ner_labels.extend(ner_output_json[i][key])
    ner_output_df.at[i, 'ner_output'] = ner_labels

#ner_output_df.to_csv('painting_ner1.csv', index=False)

#### Object Detection: Ceramics

In [ ]:
# get IIIF image urls 
object_data = pd.read_csv('/vision/ceramic_gpt.csv', na_filter=False, dtype = str)
object_data.shape

In [ ]:
# Loop through each row in the dataframe
for index, row in object_data.iterrows():
    
    # If the row already has a GPT description, skip it
    #if row['gpt_description'] != '':
        #continue
    
    print('Processing image', index + 1, 'of', len(object_data))
    # Get the time at the start of the request
    start_time = time.time()

    image_url = row['gpt_url']

    incomplete = True
    tries = 0
    while incomplete:
        tries += 1
        try:
            response = CLIENT.chat.completions.create(
              model="gpt-4-vision-preview",
              messages=[
                {
                  "role": "system", 
                  "content": "You are an intelligent and concise multi-modal model whose job is to provide specific labels for what is depicted on the surface of a 3D object."
                }, 
                {
                  "role": "user", 
                  "content": "I will provide you with several images of ceramics. For each ceramic, give me a numbered list of objects that you detect on its surface."
                },
                {
                  "role": "user",
                  "content": "Your response should only contain the list of detected objects. Each list should only contain the names of the objects, without any additional details. Skip the objects you are unsure about. Example response: '\n1. rosebud\n2. peach\n3. old man'"
                },
                {
                  "role": "user",
                  "content": [
                    {"type": "text", "text": prompt},
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": image_url,
                      },
                    },
                  ],
                }
              ],
              max_tokens=300,
            )
            incomplete = False
            #print(response)
        except Exception as e:
            # Print the error message
            print(e)
            if tries > 5:
                print('Error after 5 tries. Skipping this image.')
                break
            print('Error, waiting 10 seconds.')
            time.sleep(10)
            print('Retrying.')

    if not incomplete: # Only save the response if the request was successful
        print(image_url)
        # Extract the response from the API
        gpt_description = response.choices[0].message.content
        print(gpt_description)
        total_tokens = response.usage.total_tokens

        # Get the time at the end of the request
        end_time = time.time()
        # Calculate the elapsed time
        elapsed_time = end_time - start_time

        # Add the response to the dataframe
        object_data.at[index, 'gpt_description'] = gpt_description
        object_data.at[index, 'total_tokens'] = total_tokens
        object_data.at[index, 'elapsed_time'] = elapsed_time

        # Save the dataframe to a CSV file after each iteration in case it crashes
        object_data.to_csv('ceramic_gpt_output1.csv', index=False)

print('Done.')
    

#### Object Detection: Sculptures

In [ ]:
# get IIIF image urls 
object_data = pd.read_csv('/vision/sculpture_gpt.csv', na_filter=False, dtype = str)
object_data.shape

In [ ]:

# Loop through each row in the dataframe
for index, row in object_data.iterrows():
    
    # If the row already has a GPT description, skip it
    #if row['gpt_description'] != '':
        #continue
    
    print('Processing image', index + 1, 'of', len(object_data))
    # Get the time at the start of the request
    start_time = time.time()

    image_url = row['gpt_url']

    incomplete = True
    tries = 0
    while incomplete:
        tries += 1
        try:
            response = CLIENT.chat.completions.create(
              model="gpt-4-vision-preview",
              messages=[
                {
                  "role": "system", 
                  "content": "You are an intelligent and concise multi-modal model whose job is to provide specific labels for the objects depicted in a sculpture."
                }, 
                {
                  "role": "user", 
                  "content": "I will provide you with several images of sculptures. Give me a numbered list of objects that you detect within each sculpture."
                },
                {
                  "role": "user",
                  "content": "Your response should only contain the list of detected objects. Each list should only contain the names of the objects, without any additional details. Skip the objects you are unsure about. Example response: '\n1. Virgin Mary\n2. lamb\n3. rose'"
                },
                {
                  "role": "user",
                  "content": [
                    {"type": "text", "text": prompt},
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": image_url,
                      },
                    },
                  ],
                }
              ],
              max_tokens=300,
            )
            incomplete = False
            #print(response)
        except Exception as e:
            # Print the error message
            print(e)
            if tries > 5:
                print('Error after 5 tries. Skipping this image.')
                break
            print('Error, waiting 10 seconds.')
            time.sleep(10)
            print('Retrying.')

    if not incomplete: # Only save the response if the request was successful
        print(image_url)
        # Extract the response from the API
        gpt_description = response.choices[0].message.content
        print(gpt_description)
        total_tokens = response.usage.total_tokens

        # Get the time at the end of the request
        end_time = time.time()
        # Calculate the elapsed time
        elapsed_time = end_time - start_time

        # Add the response to the dataframe
        object_data.at[index, 'gpt_description'] = gpt_description
        object_data.at[index, 'total_tokens'] = total_tokens
        object_data.at[index, 'elapsed_time'] = elapsed_time

        # Save the dataframe to a CSV file after each iteration in case it crashes
        object_data.to_csv('sculpture_gpt_output1.csv', index=False)

print('Done.')
    

#### Object Detection: Drawings

In [ ]:
# get IIIF image urls 
# have been limited to those with creation date before 1940 or inventory number less than 1996
object_data = pd.read_csv('/vision/drawing_gpt.csv', na_filter=False, dtype = str)
object_data.shape

In [ ]:

# Loop through each row in the dataframe
for index, row in object_data.iterrows():
    
    # If the row already has a GPT description, skip it
    #if row['gpt_description'] != '':
        #continue
    
    print('Processing image', index + 1, 'of', len(object_data))
    # Get the time at the start of the request
    start_time = time.time()

    image_url = row['gpt_url']

    incomplete = True
    tries = 0
    while incomplete:
        tries += 1
        try:
            response = CLIENT.chat.completions.create(
              model="gpt-4-vision-preview",
              messages=[
                {
                  "role": "system", 
                  "content": "You are an intelligent and concise multi-modal model whose job is to provide specific labels for the main subjects of a drawing."
                }, 
                {
                  "role": "user", 
                  "content": "I will provide you with several images of drawings. Give me a numbered list of objects that you detect within each drawing."
                },
                {
                  "role": "user",
                  "content": "Your response should only contain the list of detected objects. Each list should only contain the names of the objects, without any additional details. Skip the objects you are unsure about. Example response: '\n1. olive tree\n2. flock of sheep\n3. farmhouse'"
                },
                {
                  "role": "user",
                  "content": [
                    {"type": "text", "text": prompt},
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": image_url,
                      },
                    },
                  ],
                }
              ],
              max_tokens=300,
            )
            incomplete = False
            #print(response)
        except Exception as e:
            # Print the error message
            print(e)
            if tries > 5:
                print('Error after 5 tries. Skipping this image.')
                break
            print('Error, waiting 10 seconds.')
            time.sleep(10)
            print('Retrying.')

    if not incomplete: # Only save the response if the request was successful
        print(image_url)
        # Extract the response from the API
        gpt_description = response.choices[0].message.content
        print(gpt_description)
        total_tokens = response.usage.total_tokens

        # Get the time at the end of the request
        end_time = time.time()
        # Calculate the elapsed time
        elapsed_time = end_time - start_time

        # Add the response to the dataframe
        object_data.at[index, 'gpt_description'] = gpt_description
        object_data.at[index, 'total_tokens'] = total_tokens
        object_data.at[index, 'elapsed_time'] = elapsed_time

        # Save the dataframe to a CSV file after each iteration in case it crashes
        object_data.to_csv('drawing_gpt_output1.csv', index=False)

print('Done.')
    